# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lethem,3.3803,-59.7968,92.80,45,98,8.03,GY,1711821296
1,1,adamstown,-25.0660,-130.1015,73.62,68,100,28.25,PN,1711821296
2,2,afaahiti,-17.7500,-149.2833,82.27,83,89,22.62,PF,1711821296
3,3,port-aux-francais,-49.3500,70.2167,43.03,94,100,16.22,TF,1711821297
4,4,kupang,-10.1667,123.5833,79.38,100,100,2.30,ID,1711821297


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
# Configure the map plot
# YOUR CODE HERE

humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Humidity"],
    alpha = .5
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[(city_data_df["Humidity"] < 75) \
                    & (city_data_df["Max Temp"] > 60) \
                    & (city_data_df["Max Temp"] < 75) \
                    & (city_data_df["Cloudiness"] < 50)]


#Change these to Fahrenhiet

# Drop any rows with null values
ideal_city_data_clean = ideal_city_data_df.dropna()



# Display sample data
ideal_city_data_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,hermanus,-34.4187,19.2345,74.80,68,26,10.56,ZA,1711821300
47,47,taltal,-25.4000,-70.4833,71.87,74,25,6.87,CL,1711821307
81,81,frostburg,39.6581,-78.9284,63.91,38,40,1.99,US,1711821134
90,90,richards bay,-28.7830,32.0377,72.50,72,41,6.96,ZA,1711821319
162,162,tindouf,27.6711,-8.1474,74.37,33,1,21.09,DZ,1711821335


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_clean.copy()
hotel_df = hotel_df.drop(["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"], axis = 1)


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
17,hermanus,-34.4187,19.2345,68,ZA,
47,taltal,-25.4000,-70.4833,74,CL,
81,frostburg,39.6581,-78.9284,38,US,
90,richards bay,-28.7830,32.0377,72,ZA,
162,tindouf,27.6711,-8.1474,33,DZ,
204,allen,41.0706,-85.0144,71,US,
206,hamilton,39.1834,-84.5333,44,US,
242,at taj,24.2049,23.2857,7,LY,
253,lake havasu city,34.4839,-114.3224,19,US,
255,abra pampa,-22.7205,-65.6970,37,AR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":1
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    #print(params)
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

     # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    #Convert the API response to JSON format
    name_address = name_address.json()  

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hermanus - nearest hotel: Aloe guest house
taltal - nearest hotel: Hostería Taltal
frostburg - nearest hotel: Hotel Gunter
richards bay - nearest hotel: SUN1 Richards Bay
tindouf - nearest hotel: محمد بوسبي
allen - nearest hotel: Holiday Inn Express Fort Wayne-East (New Haven)
hamilton - nearest hotel: North Vista Manor
at taj - nearest hotel: No hotel found
lake havasu city - nearest hotel: The Sands Vacation Resort
abra pampa - nearest hotel: Residencia El Norte
tura - nearest hotel: Hotel Polo Orchid
ocean city - nearest hotel: Watson's Regency Suites
al burayqah - nearest hotel: فندق ماس ليبيا
quthing - nearest hotel: Mountainside Hotel
sucre - nearest hotel: El Gobernador Hostal
guatrache - nearest hotel: No hotel found
la paz - nearest hotel: Alojamiento Tokio
pacific grove - nearest hotel: Pacific Grove Inn
dubrovnik - nearest hotel: Bellevue
opava - nearest hotel: Hotel Koruna
mar del plata - nearest hotel: Gran hotel Mónaco
coronel dorrego - nearest hotel

,City,Lat,Lng,Humidity,Country,Hotel Name
17,hermanus,-34.4187,19.2345,68,ZA,Aloe guest house
47,taltal,-25.4000,-70.4833,74,CL,Hostería Taltal
81,frostburg,39.6581,-78.9284,38,US,Hotel Gunter
90,richards bay,-28.7830,32.0377,72,ZA,SUN1 Richards Bay
162,tindouf,27.6711,-8.1474,33,DZ,محمد بوسبي
204,allen,41.0706,-85.0144,71,US,Holiday Inn Express Fort Wayne-East (New Haven)
206,hamilton,39.1834,-84.5333,44,US,North Vista Manor
242,at taj,24.2049,23.2857,7,LY,No hotel found
253,lake havasu city,34.4839,-114.3224,19,US,The Sands Vacation Resort
255,abra pampa,-22.7205,-65.6970,37,AR,Residencia El Norte


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
    alpha = .5
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)